In [1]:
import pandas as pd
import plotly.express as px
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB

# Import des données


In [3]:
df = pd.read_csv(
    "../DataSetBrut/SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)
df

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


On cherche à modéliser la probabilité qu'un message soit un spam à partir de son contenu. Il nous faut donc un modèle de classification binaire. On peux donc utiliser un modèle linaire SVC, ou un naive bayes.

# Prétraitement des données

Je vais d'abord binariser la variable cible, sous forme d'un entier (0 ou 1).

In [3]:
df["spam"] = df["spam"].map({'spam': 1, 'ham': 0}).astype(int)


In [4]:
df

,spam,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


# Approche par constructions de feature descriptives des messages

## Création des features

On vérifie la présence d'un numéro de téléphone.

In [5]:
df['has_phone_number'] = df['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
df

,spam,text,has_phone_number
0,0,"Go until jurong point, crazy.. Available only ...",0
1,0,Ok lar... Joking wif u oni...,0
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,0
3,0,U dun say so early hor... U c already then say...,0
4,0,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,1
5568,0,Will ü b going to esplanade fr home?,0
5569,0,"Pity, * was in mood for that. So...any other s...",0
5570,0,The guy did some bitching but I acted like i'd...,0


On vérifie la présence de monnaies.

In [6]:
df['has_currency_symbol'] = df['text'].apply(lambda x: 1 if re.search(r'[\$\€\£]', x) else 0)
df

,spam,text,has_phone_number,has_currency_symbol
0,0,"Go until jurong point, crazy.. Available only ...",0,0
1,0,Ok lar... Joking wif u oni...,0,0
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,0,0
3,0,U dun say so early hor... U c already then say...,0,0
4,0,"Nah I don't think he goes to usf, he lives aro...",0,0
...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,1,1
5568,0,Will ü b going to esplanade fr home?,0,0
5569,0,"Pity, * was in mood for that. So...any other s...",0,0
5570,0,The guy did some bitching but I acted like i'd...,0,0


In [7]:
df[df["spam"] == 1]["text"].iloc[0]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

On calcule la longueur des messages.

In [8]:
df['message_length'] = df['text'].apply(len)
df

,spam,text,has_phone_number,has_currency_symbol,message_length
0,0,"Go until jurong point, crazy.. Available only ...",0,0,111
1,0,Ok lar... Joking wif u oni...,0,0,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,0,0,155
3,0,U dun say so early hor... U c already then say...,0,0,49
4,0,"Nah I don't think he goes to usf, he lives aro...",0,0,61
...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,1,1,160
5568,0,Will ü b going to esplanade fr home?,0,0,36
5569,0,"Pity, * was in mood for that. So...any other s...",0,0,57
5570,0,The guy did some bitching but I acted like i'd...,0,0,125


## Entrainement et évaluation du modèle

In [9]:

X = df[['message_length', 'has_phone_number', 'has_currency_symbol']]
y = df['spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      1.00      0.97       966
           1       0.97      0.61      0.75       149

    accuracy                           0.95      1115
   macro avg       0.96      0.80      0.86      1115
weighted avg       0.95      0.95      0.94      1115



# Approche par Vectorisation de text (NLP)

Il faut donc convertir le texte en une représentation numérique. J'utilise pour cela TfidfVectorizer.  C'est une technique de pondération des mots utilisée dans la récupération d'informations et le traitement du langage naturel (NLP). Elle évalue l'importance d'un mot dans un document par rapport à une collection de documents (corpus).
- TF (Term Frequency) : Nombre de fois qu'un mot apparaît dans un document, divisé par le nombre total de mots dans ce document.
- IDF (Inverse Document Frequency) : Mesure l'importance d'un mot en tenant compte de sa fréquence dans tout le corpus. Moins un mot est fréquent dans les autres documents, plus sa valeur IDF est élevée.

In [10]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['text'])
y = df['spam']


In [11]:
df['has_phone_number'] = df['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
df

,spam,text,has_phone_number,has_currency_symbol,message_length
0,0,"Go until jurong point, crazy.. Available only ...",0,0,111
1,0,Ok lar... Joking wif u oni...,0,0,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,0,0,155
3,0,U dun say so early hor... U c already then say...,0,0,49
4,0,"Nah I don't think he goes to usf, he lives aro...",0,0,61
...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,1,1,160
5568,0,Will ü b going to esplanade fr home?,0,0,36
5569,0,"Pity, * was in mood for that. So...any other s...",0,0,57
5570,0,The guy did some bitching but I acted like i'd...,0,0,125


In [12]:
pd.concat([pd.DataFrame(X.toarray()), df["message_length"]], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,8435,8436,8437,8438,8439,8440,8441,8442,8443,message_length
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,155
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125


## Utilisation du modèle naive bayes

On sépare les données d'entrainement et de tests.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


On entraine le modèle.

In [14]:
model = MultinomialNB()
model.fit(X_train, y_train)


MultinomialNB()

On évalue le modèle sur les données de test.

In [15]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.979372197309417
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.85      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



Avec la méthode de cross validation : 

In [16]:
print(cross_val_score(model, X, y, cv=3))

[0.967169  0.9633818 0.9633818]


Le modèle est assez performant mais 25% des vrais spams n'ont pas été détectés (faux négatifs), donc non suffisant.

## Utilisation du modèle linéraire SVC

On sépare les données d'entrainement et de tests.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


On entraine le modèle.

In [18]:
model = LinearSVC()
model.fit(X_train, y_train)


LinearSVC()

On évalue le modèle sur les données de test.

In [19]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9802690582959641
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.99      0.86      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



Avec la méthode de cross validation :

In [20]:
print(cross_val_score(model, X, y, cv=3))

[0.9795479  0.97576737 0.97630587]


On obtient une précision plutot élevée, avec une accuracy de 98%, même si certains spams sont toujours manqués.

In [21]:
new_messages = ['You won 200 billion dollars, call now!', 'Hi, how are you?']
X_new = vectorizer.transform(new_messages)
predictions = model.predict(X_new)
print(predictions)


[0 0]


In [22]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

documents = df["text"]
labels = df["spam"] 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)
model = LinearSVC()
model.fit(X, labels)
terms = vectorizer.get_feature_names_out()
coefficients = model.coef_.flatten()
term_weights = pd.DataFrame({
    'term': terms,
    'weight': coefficients
})
term_weights_sorted = term_weights.sort_values(by='weight', ascending=False)
term_weights_sorted.head(30)


,term,weight
346,146tf150p,1.438561
6526,ringtoneking,0.863136
776,84484,0.863136
6525,ringtone,0.846111
25,07090201529,0.701952
8016,uk,0.670268
802,88066,0.652444
7327,stories,0.616624
3219,filthy,0.616624
7986,txt,0.583218


In [23]:
import plotly.express as px
px.violin(term_weights_sorted[(term_weights_sorted["weight"] > -0.5) & (term_weights_sorted["weight"] < 0.5)], y="weight")

In [24]:
term_weights_sorted[term_weights_sorted["term"] == "rate"]

,term,weight
6284,rate,0.33898


In [25]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)

df['has_phone_number'] = df['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
df['has_currency_symbol'] = df['text'].apply(lambda x: 1 if re.search(r'[\$\€\£]', x) else 0)
df['message_length'] = df['text'].apply(len)


vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['text'])
df_vectorized = pd.concat([pd.DataFrame(X.toarray()), df["message_length"], df["has_phone_number"], df["has_currency_symbol"]], axis=1)

df_vectorized

,0,1,2,3,4,5,6,7,8,9,...,8437,8438,8439,8440,8441,8442,8443,message_length,has_phone_number,has_currency_symbol
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,155,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160,1,1
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,0,0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57,0,0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125,0,0


In [26]:
df["spam"]

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: spam, Length: 5572, dtype: object

## Test KNN

In [27]:
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [28]:
df = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)
df['has_phone_number'] = df['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
df['has_currency_symbol'] = df['text'].apply(lambda x: 1 if re.search(r'[\$\€\£]', x) else 0)
df["has_special_characters"] = df['text'].apply(lambda x: 1 if re.search(r'[!@#$%^&*(),.?":{}|<>]', x) else 0)
df['message_length'] = df['text'].apply(len)
df["uppercase_proportion"] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
df["has_url"] = df['text'].apply(lambda x: 1 if re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x) else 0)
df["non_alpha_charaters_proportion"] = df['text'].apply(lambda x: sum(1 for c in x if not c.isalpha()) / len(x) if len(x) > 0 else 0)
df

,spam,text,has_phone_number,has_currency_symbol,has_special_characters,message_length,uppercase_proportion,has_url,non_alpha_charaters_proportion
0,ham,"Go until jurong point, crazy.. Available only ...",0,0,1,111,0.027027,0,0.252252
1,ham,Ok lar... Joking wif u oni...,0,0,1,29,0.068966,0,0.379310
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,0,1,155,0.064516,0,0.374194
3,ham,U dun say so early hor... U c already then say...,0,0,1,49,0.040816,0,0.326531
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,0,1,61,0.032787,0,0.229508
...,...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,1,1,160,0.056250,0,0.368750
5568,ham,Will ü b going to esplanade fr home?,0,0,1,36,0.027778,0,0.222222
5569,ham,"Pity, * was in mood for that. So...any other s...",0,0,1,57,0.035088,0,0.280702
5570,ham,The guy did some bitching but I acted like i'd...,0,0,0,125,0.016000,0,0.208000


In [29]:
new_columns = [
    'has_phone_number',
    'has_currency_symbol',
    'has_special_characters',
    'message_length',
    'uppercase_proportion',
    'has_url',
    'non_alpha_charaters_proportion'
]

In [30]:
X = df[new_columns]
y = df["spam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.95      0.99      0.97       966
        spam       0.93      0.63      0.75       149

    accuracy                           0.94      1115
   macro avg       0.94      0.81      0.86      1115
weighted avg       0.94      0.94      0.94      1115



In [31]:
new_messages = pd.DataFrame(['You won 200 billion dollars, call now or click here to claim your prise !', 'Hi, how are you?'], columns=["text"])
new_messages['has_phone_number'] = new_messages['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
new_messages['has_currency_symbol'] = new_messages['text'].apply(lambda x: 1 if re.search(r'[\$\€\£]', x) else 0)
new_messages["has_special_characters"] = new_messages['text'].apply(lambda x: 1 if re.search(r'[!@#$%^&*(),.?":{}|<>]', x) else 0)
new_messages['message_length'] = new_messages['text'].apply(len)
new_messages["uppercase_proportion"] = new_messages['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
new_messages["has_url"] = new_messages['text'].apply(lambda x: 1 if re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x) else 0)
new_messages["non_alpha_charaters_proportion"] = new_messages['text'].apply(lambda x: sum(1 for c in x if not c.isalpha()) / len(x) if len(x) > 0 else 0)
X_new = new_messages[['has_phone_number', 'has_currency_symbol', 'has_special_characters', 'message_length', 'uppercase_proportion', 'has_url', 'non_alpha_charaters_proportion']]
predictions = model.predict(X_new)
print(predictions)

['ham' 'ham']


## Decision Tree

In [32]:
import pandas as pd
import re
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [33]:
df = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)
df['has_phone_number'] = df['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
df['has_currency_symbol'] = df['text'].apply(lambda x: 1 if re.search(r'[\$\€\£]', x) else 0)
df["has_special_characters"] = df['text'].apply(lambda x: 1 if re.search(r'[!@#$%^&*(),.?":{}|<>]', x) else 0)
df['message_length'] = df['text'].apply(len)
df["uppercase_proportion"] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
df["has_url"] = df['text'].apply(lambda x: 1 if re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x) else 0)
df["non_alpha_charaters_proportion"] = df['text'].apply(lambda x: sum(1 for c in x if not c.isalpha()) / len(x) if len(x) > 0 else 0)
df

,spam,text,has_phone_number,has_currency_symbol,has_special_characters,message_length,uppercase_proportion,has_url,non_alpha_charaters_proportion
0,ham,"Go until jurong point, crazy.. Available only ...",0,0,1,111,0.027027,0,0.252252
1,ham,Ok lar... Joking wif u oni...,0,0,1,29,0.068966,0,0.379310
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,0,1,155,0.064516,0,0.374194
3,ham,U dun say so early hor... U c already then say...,0,0,1,49,0.040816,0,0.326531
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,0,1,61,0.032787,0,0.229508
...,...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,1,1,160,0.056250,0,0.368750
5568,ham,Will ü b going to esplanade fr home?,0,0,1,36,0.027778,0,0.222222
5569,ham,"Pity, * was in mood for that. So...any other s...",0,0,1,57,0.035088,0,0.280702
5570,ham,The guy did some bitching but I acted like i'd...,0,0,0,125,0.016000,0,0.208000


In [34]:
new_columns = [
    'has_phone_number',
    'has_currency_symbol',
    'has_special_characters',
    'message_length',
    'uppercase_proportion',
    'has_url',
    'non_alpha_charaters_proportion'
]

In [35]:
X = df[new_columns]
y = df["spam"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.97      0.97      0.97       966
        spam       0.83      0.79      0.81       149

    accuracy                           0.95      1115
   macro avg       0.90      0.88      0.89      1115
weighted avg       0.95      0.95      0.95      1115



In [36]:
new_messages = pd.DataFrame(['You won 200 billion dollars, call now at 0256547856 or click here to claim your prise : www.google.com !', 'Hi, how are you?'], columns=["text"])
new_messages['has_phone_number'] = new_messages['text'].apply(lambda x: 1 if re.search(r'\b\d{10,}\b', x) else 0)
new_messages['has_currency_symbol'] = new_messages['text'].apply(lambda x: 1 if re.search(r'[\$\€\£]', x) else 0)
new_messages["has_special_characters"] = new_messages['text'].apply(lambda x: 1 if re.search(r'[!@#$%^&*(),.?":{}|<>]', x) else 0)
new_messages['message_length'] = new_messages['text'].apply(len)
new_messages["uppercase_proportion"] = new_messages['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
new_messages["has_url"] = new_messages['text'].apply(lambda x: 1 if re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x) else 0)
new_messages["non_alpha_charaters_proportion"] = new_messages['text'].apply(lambda x: sum(1 for c in x if not c.isalpha()) / len(x) if len(x) > 0 else 0)
X_new = new_messages[['has_phone_number', 'has_currency_symbol', 'has_special_characters', 'message_length', 'uppercase_proportion', 'has_url', 'non_alpha_charaters_proportion']]
predictions = model.predict(X_new)
print(predictions)

['spam' 'ham']
